# LSA Stage

The previous stages of the processing pipeline involved splitting an audio file from a corpus entry into speech segements by applying a VAD algorithm and then generating transcripts for these segments by using an RNN that was trained with CTC. Provided both stages produce reasonably good output, this should result in one transcript per speech segment. Those transcripts can now be aligned with the (given) transcript of the whole corpus entry by searching each speech transcript within the whole transcript, i.e. find a partially matching substring $b$ within a longer substring $a$. This process is called _Local Sequence Alignment (LSA)_.

There are two main challenges with LSA:

1. string $b$ matches some substring of $a$ only **partially**, i.e. the similarities between $b$ and a substring of $a$ must be measured somehow
1. the alignment is **local** i.e. a very large number of all possible substrings of various lengths need to be compared to find the optimal alignment.

## Smith-Waterman algorithm

The [Smith-Waterman algorithm](https://en.wikipedia.org/wiki/Smith%E2%80%93Waterman_algorithm) performs LSA for two string $a$ and $b$ with $$\bigl|a\bigr| = m$$ and $$\bigl|b\bigr| = n$$ in $\mathcal{O}(mn)$. The similarities of alignments are calculated by creating a _scoring matrix_. The optimal alignment is then calculated by maximizing the similarity over all alignments. This involves backtracing in the scoring matrix, starting with the maximal similarity value. Since the maximal value may be present several times in the scoring matrix, there may also be several possible optimal alignments of string $b$ with $a$.

Both steps of the algorithm are explained in a bit more depth in the following paragraphs. For a more detailled explanation, I recommend reading the [Wikipedia article](https://en.wikipedia.org/wiki/Smith%E2%80%93Waterman_algorithm).

### Step 1: Scoring matrix

The scoring matrix $H$ will have dimensions $(m \times n)$. The first axis corresponds to the letters in string $a$, the second axis to the letters in string $b$. The values of $H$ are calculated by recursively finding the maximum similarity between substrings of $a$ and $b$. For this, two values need ot be defined:

* $f$: cost for inserting a gap or deleting a value in $b$
* $s(a_i, b_j)$: score function that compares two letters $a_i$ and $b_j$ from $a$ resp. $b$ and calculates a score for their similarity

$s(a_i, b_j)$ is usually implemented as assigning a positive value if the two letters are identical and a negative value if the letters are different. The value $H_{ij}$ of a cell in the scoring matrix can be calculated by comparing the values in various directions. This produces four values:

* **diagonal direction**: the similarity is calculated by adding the similarity score of the next letters to the similarity scores of the substring prefix)
* **vertical direction**: the similarity is calculated by subtracting cost $f$ from the cell value above $H_{ij}$. This corresponds to inserting a gap in $b$.
* **horizontal direction**: the similarity is calculated by subtracting cost $f$ from the cell value to the left of $H_{ij}$. This corresponds to deleting the letter in $b$.
* **constant value $0$**: This corresponds to _no similarity_, i.e. the two substrings are not similar at all.

The cell value is then set as the maximum of these values. This means $H_{ij} \ge 0$ for each entry in $H$. For simplicity, the same gap cost is used for gaps of various lengths (fixed gap cost). However, for other purposes the gap cost could also grow linearly with the gap length.

The following code calculates this matrix for two strings a and b with linear gap costs. For performance reasons I went for an implementation with NumPy arrays. Values for match scores and gap costs can be changed. The default values correspond to the example from Wikipedia:

```python
import itertools
import numpy as np

def matrix(a, b, match_score=3, gap_cost=2):
    H = np.zeros((len(a) + 1, len(b) + 1), np.int)

    for i, j in itertools.product(range(1, H.shape[0]), range(1, H.shape[1])):
        match = H[i - 1, j - 1] + (match_score if a[i - 1] == b[j - 1] else - match_score)
        delete = H[i - 1, j] - gap_cost
        insert = H[i, j - 1] - gap_cost
        H[i, j] = max(match, delete, insert, 0)
    return H
```

### Step 2: Backtracking

The optimal alignment can be found by starting with the maximal value $v_{max}$ in $H$, i.e. the score of the substring $b'$ with maximum similarity for an alignment in $a$. Note that $b'$ is a variant of $b$ including gaps and deleted letters. The value and position of substring $b'$ can be constructed by backtracing from its similarity score. This is done by starting with the cell containing $v_{max}$ at coordinates $(i, j)$. Backtracing is done by assigning $b'$ the empty string and then recursively performing the following steps:

1. find coordinate $j$ of the cell with the maximal value in the last row $i$
2. distinguish the following cases:    
    a. $j$ changed by more than $1$ (i.e. move left in the matrix): this means a letter from $b$ was skipped. In this case simply prepend $b_j$ to $b'$
    a. $j$ did not change (i.e. move up in the matrix because the maximal value is at the same horizontal position as before) this means $b'$ contains a gap. In this case prepend $b_j$ to $b'$ and then $-$ to mark the gap
3. delete the last row from $H$ and start over
    
The recursion stops when a value of $H_{ij} = 0$ is encountered. The value of $j$ marks the starting position (i.e. the alignment) of substring $b'$ within $a$.

The following Python script is an implementation of the backtracing step.

```python
def traceback(H, b, b_='', old_i=0):
    # flip H to get index of **last** occurrence of H.max() with np.argmax()
    H_flip = np.flip(np.flip(H, 0), 1)
    i_, j_ = np.unravel_index(H_flip.argmax(), H_flip.shape)
    i, j = np.subtract(H.shape, (i_ + 1, j_ + 1))  # (i, j) are **last** indexes of H.max()
    if H[i, j] == 0:
        return b_, j
    b_ = b[j - 1] + '-' + b_ if old_i - i > 1 else b[j - 1] + b_
    return traceback(H[0:i, 0:j], b, b_, i)
```

### Examples

Execute the following cells to see some examples created with above implementation of the Smith-Waterman algorithm.

In [1]:
from tabulate import tabulate
from smith_waterman import smith_waterman

examples = [
    ('Mississippi', 'issp'),
    ('schöner Götterfunken', 'schönner Goterfunken')
]
table = []
for a, b in examples:
    start, end, b_ = smith_waterman(a, b)
    table.append([a, b, b_, start, end, a[:start] + '[' + a[start:end] + ']' + a[end:]])

print(tabulate(table, headers=['a', 'b', 'b\'', 'start', 'end', 'alignment']))

a                     b                     b'                      start    end  alignment
--------------------  --------------------  --------------------  -------  -----  ----------------------
Mississippi           issp                  iss-p                       4      9  Miss[issip]pi
schöner Götterfunken  schönner Goterfunken  schoner got-erfunken        0     20  [schöner Götterfunken]


## Ceiling analysis

Evaluation of the [VAD stage](02_vad.ipynb) has shown that WebRTC works very well and is able to create not too long speech segments from arbitrary audio. Experimental results of the [ASR-](04_asr.ipynb) have shown that the ability of an RNN trained with CTC will depend heavily on the amount of training data and the chose model architecture. 

Due to time constraint, training a fully-fletched RNN with acceptable performance was not possible. However, since the overall performance of the system will essentially depend on the results of this stage, a ceiling analysis is performed. This means that the end result (the output of the LSA stage) can be estimated by simulating near-perfect input from the ASR-stage. 

This is done by collecting transcripts from training sessions where the RNN was trained on only a very small subset of the actual training data. Since the RNN will then overfit to this data, the RNN is not usable in production. However, because a LER-rate of a few percent was deemed acceptable, the training output resembles some near-perfect transcript that can be used to analyze the behavior in the LSA-stage.

The following situations need to be analyzed:

* **missing parts in transcript**: How does the LSA stage handle transcripts of speech segments that do not contain all words
* **spontaneous sounds**: 